### set libraly

In [42]:
import pandas as pd
import os
import numpy as np
import glob
import codecs


In [51]:
pd.set_option('display.max_rows', 5000)


### set path

In [2]:
input = "Input\\"
input_data = "Input\\data\\"
input_mapping = "Input\\mapping\\"
output = "Output\\"
dataprepare = "dataprepare\\"
#os.mkdir(input)
#os.mkdir(output)
#os.mkdir(input_data)
#os.mkdir(input_mapping)
#os.mkdir(dataprepare)

### prepare data

In [3]:
class RawData:
      def __init__(self, folder, folder2):
    
        self.folder = folder
        self.file = glob.glob(folder+"/*.csv")[-1]
        self.folder2 = folder2
        self.file2 = glob.glob(folder+"/*.xlsb")[-1]

    
        print(f"Start to read CSV file '{self.file[-26:]}' in folder '{self.folder}'")
        self.df = pd.read_csv(self.file,encoding='UTF-8' )
        print(f"Read CSV file '{self.file[-22:]}' done!","\n")
        
      def remove_white_space(self):
            print("Start remove_white_space function")
            for col in self.df.columns.tolist():
              if self.df[col].dtypes == object:
                self.df[col] = self.df[col].str.strip()
              print("Remove_white_space done!","\n")
      def save_csv(self):
              self.df.to_csv(input_data+"test.csv",encoding='TIS-620')
              print(f"save file {self.file[-22:]} done" , "\n")

      #def mapping_data(self):
       #       self.df_mapping = pd.read_excel(self.file2,encoding='UTF-8' )
              
    #self.file[-22:]

In [5]:
test = RawData(input_data)
test.remove_white_space()

Start to read CSV file 'Input\data\CRR2563.csv' in folder 'Input\data\'
Read CSV file 'Input\data\CRR2563.csv' done! 

Start remove_white_space function
Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 

Remove_white_space done! 



## manual

In [3]:
mapping = pd.read_excel(input_mapping+'RPISIC01_20211231_ข้อมูลรายตัวสำหรับตรวจสอบ (22 ประเภทธุรกิจ).xlsb',engine='pyxlsb',usecols=['Customer Number','Account Number','USER2','Account Class','PF/NPF Flag'] )

In [8]:
### prepare
data = pd.read_csv(input_data+'CRR2563.csv',encoding='UTF-8')
data['เลขที่ CIF'] = data['เลขที่ CIF'].str.strip()
data.dropna(subset=['เลขที่ CIF'],inplace=True)
data.rename(columns={"เลขที่ CIF": "Customer Number"},inplace=True)

In [14]:
### fitter not number
data_main = data[data['Customer Number'].str.contains(r'[0-9]')]
### convert type object to int
data_main['Customer Number'] = data_main['Customer Number'].astype(np.int64)

In [24]:
df = data_main.merge(mapping,on='Customer Number',how='left')

In [36]:
df['USER2'].isnull().value_counts()

,บุคคล/นิติบุคคลที่ถูกประเมิน,Customer Number,วันที่ประเมิน,Grade,ประเภท,วัตถุประสงค์,Account Number,Account Class,PF/NPF Flag
USER2,,,,,,,,,
B1,2544,2544,2544,2542,2544,2258,2544,2515,2544
B2,182,182,182,182,182,172,182,182,182
B3,204,204,204,204,204,192,204,204,204
B4,54,54,54,54,54,35,54,54,54
B5,42,42,42,42,42,31,42,42,42


In [75]:
df = df.dropna(subset=['USER2'])
df = df.drop_duplicates(subset=['Customer Number'])
df = df.dropna(subset=['Grade'])
df = df[~(df['Grade']=='-')]
df.reset_index(drop=True)

### Define Good_Bad

In [248]:
def define_Good_Bad(df):
    condition = [(df['USER2'] == 'B1')|(df['USER2'] == 'B2'),(df['USER2'] == 'B2')|(df['USER2'] == 'B4')|(df['USER2'] == 'B5')]
    mapping = (1,0)
    df['Good_Bad'] = np.select(condition, mapping)
    

In [249]:
define_Good_Bad(df)
#df.drop(columns=['Good','Bad'],inplace=True)

In [258]:
df.head(1)

,บุคคล/นิติบุคคลที่ถูกประเมิน,Customer Number,วันที่ประเมิน,Grade,ประเภท,วัตถุประสงค์,Account Number,USER2,Account Class,PF/NPF Flag,Good_Bad
0,บริษัท บุญบารมีเมตตา พร็อพเพอร์ตี้ จำกัด,540004526,30/4/2563,C+,M & L Services,เปลี่ยนแปลงเงื่อนไข,8.989472e+09,B2,2.0,1_PF,1


### Medium && Large CRR

In [316]:
df['Grade'].isna().value_counts()
df.groupby(['Grade']).count()
df_CRR = df[~df['Grade'].str.contains('[s,S]')]
df_CRR.groupby(['Grade','Good_Bad']).count()

บุคคล/นิติบุคคลที่ถูกประเมิน  Customer Number  วันที่ประเมิน  \
Grade Good_Bad                                                                 
A+    0                                    1                1              1   
      1                                    5                5              5   
B+    1                                   29               29             29   
BB    1                                   22               22             22   
BBB   1                                   22               22             22   
C+    0                                    1                1              1   
      1                                   28               28             28   
CC    1                                   16               16             16   
CCC   0                                    2                2              2   
      1                                   17               17             17   
D+    1                                    1                1              1   
DD    1                                    1                1              1   
DDD   0                                    2                2              2   
      1                                   10               10             10   

                ประเภท  วัตถุประสงค์  Account Number  USER2  Account Class  \
Grade Good_Bad                                                               
A+    0              1             1               1      1              1   
      1              5             5               5      5              5   
B+    1             29            29              29     29             27   
BB    1             22            22              22     22             21   
BBB   1             22            22              22     22             22   
C+    0              1             1               1      1              1   
      1             28            28              28     28             27   
CC    1             16            16              16     16             16   
CCC   0              2             2               2      2              2   
      1             17            17              17     17             17   
D+    1              1             1               1      1              0   
DD    1              1             1               1      1              1   
DDD   0              2             2               2      2              2   
      1             10            10              10     10             10   

                PF/NPF Flag  
Grade Good_Bad               
A+    0                   1  
      1                   5  
B+    1                  29  
BB    1                  22  
BBB   1                  22  
C+    0                   1  
      1                  28  
CC    1                  16  
CCC   0                   2  
      1                  17  
D+    1                   1  
DD    1                   1  
DDD   0                   2  
      1                  10

### Small CRR

In [319]:
df_sCRR = df[df['Grade'].str.contains('[s,S]')]
df_sCRR.loc[df_sCRR['Grade']=='SBB','Grade'] = 'sBB'
df_sCRR.loc[df_sCRR['Grade']=='sc+','Grade'] = 'sC+'
df_sCRR = df_sCRR[~(df_sCRR['Grade']=='SS')]
df_sCRR.groupby(['Grade']).count()
df_CRR.groupby(['Grade','Good_Bad']).count()

c:\Users\630039\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


บุคคล/นิติบุคคลที่ถูกประเมิน  Customer Number  วันที่ประเมิน  \
Grade Good_Bad                                                                 
A+    0                                    1                1              1   
      1                                    5                5              5   
B+    1                                   29               29             29   
BB    1                                   22               22             22   
BBB   1                                   22               22             22   
C+    0                                    1                1              1   
      1                                   28               28             28   
CC    1                                   16               16             16   
CCC   0                                    2                2              2   
      1                                   17               17             17   
D+    1                                    1                1              1   
DD    1                                    1                1              1   
DDD   0                                    2                2              2   
      1                                   10               10             10   

                ประเภท  วัตถุประสงค์  Account Number  USER2  Account Class  \
Grade Good_Bad                                                               
A+    0              1             1               1      1              1   
      1              5             5               5      5              5   
B+    1             29            29              29     29             27   
BB    1             22            22              22     22             21   
BBB   1             22            22              22     22             22   
C+    0              1             1               1      1              1   
      1             28            28              28     28             27   
CC    1             16            16              16     16             16   
CCC   0              2             2               2      2              2   
      1             17            17              17     17             17   
D+    1              1             1               1      1              0   
DD    1              1             1               1      1              1   
DDD   0              2             2               2      2              2   
      1             10            10              10     10             10   

                PF/NPF Flag  
Grade Good_Bad               
A+    0                   1  
      1                   5  
B+    1                  29  
BB    1                  22  
BBB   1                  22  
C+    0                   1  
      1                  28  
CC    1                  16  
CCC   0                   2  
      1                  17  
D+    1                   1  
DD    1                   1  
DDD   0                   2  
      1                  10